In [ ]:
# IMPORTANT: Insert Project Token before running this notebook

### Predicting Customer Churn

This notebook performs these steps:<br/>
1. Builds a spark-ml model to predict customer churn
2. Stores the model into the project
3. Stores the model into the deployment space
4. Creates and test an Online Deployement for the model


### Environment Setup 

In [ ]:
import json, requests
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
# Update WML API, which will be used for saving the model in the project
!pip install -U ibm-watson-machine-learning

### Step 1: Load data 

In [ ]:
import ibmos2spark
# @hidden_cell

# IMPORTANT: replace with your COS API key
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-63921daf-f1f3-4aa6-b49d-6880253d6f67',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': 'replace-with-your-cos-api'
}

configuration_name = 'os_1e498447d5f74cd6b90b92b35bb6514e_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# IMPORTANT: replace the cos.url value with the BUCKET value for your COS
customer_churn = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option("inferSchema", "true")\
  .load(cos.url('churn.csv', 'replace-with-your-bucket-value'))
customer_churn.take(5)

In [ ]:
# IMPORTANT: replace the cos.url value with the BUCKET value for your COS
customer = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option("inferSchema", "true")\
  .load(cos.url('customer-profile.csv', 'replace-with-your-bucket-value'))

customer.take(5)

### Step 2: Merge Files

In [ ]:
data=customer.join(customer_churn,customer['ID']==customer_churn['ID']).select(customer['*'],customer_churn['CHURN'])

### Step 3: Rename some columns
This step is to remove spaces from columns names, it's an example of data preparation that you may have to do before creating a model. 

In [ ]:
data = data.withColumnRenamed("Est Income", "EstIncome").withColumnRenamed("Car Owner","CarOwner")
data.toPandas().head()

In [ ]:
data.toPandas().shape

In [ ]:
data.printSchema()

### Step 4: Data understanding

In [ ]:
df = data.toPandas()

In [ ]:
g1 = sns.countplot(data=df, x='CHURN', order=df.CHURN.value_counts().index)
plt.title('Customer Churn Rates')
plt.ylabel('Count of Churn')
plt.ylim(0, 800)
#Add percentages to the graph
total = float(len(df)) #one person per row
for p in g1.patches:
    height = p.get_height()
    g1.text(p.get_x()+p.get_width()/2.,
            height + 1,
            '{0:.0%}'.format(height/total),
            ha="center") 
plt.show()

In [ ]:
sns.catplot(x="CHURN", y="AvgMonthlySpend",
                 hue="MembershipPlan", col="Paymethod",
                 data=df, kind='box',
                 height=7, aspect=.81);

### Step 5: Build the Spark pipeline and the Random Forest model
"Pipeline" is an API in SparkML that's used for building models.
Additional information on SparkML: https://spark.apache.org/docs/2.1.0/ml-guide.html

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer, IndexToString
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

# Prepare string variables so that they can be used by the decision tree algorithm
# StringIndexer encodes a string column of labels to a column of label indices
SI1 = StringIndexer(inputCol='Gender', outputCol='GenderEncoded')
SI2 = StringIndexer(inputCol='Status',outputCol='StatusEncoded')
SI3 = StringIndexer(inputCol='CarOwner',outputCol='CarOwnerEncoded')
SI4 = StringIndexer(inputCol='Paymethod',outputCol='PaymethodEncoded')
SI5 = StringIndexer(inputCol='MembershipPlan',outputCol='MembershipPlanEncoded')

labelIndexer = StringIndexer(inputCol='CHURN', outputCol='label').fit(data)

# Pipelines API requires that input variables are passed in  a vector
assembler = VectorAssembler(inputCols=["GenderEncoded", "StatusEncoded", "CarOwnerEncoded", "PaymethodEncoded", "MembershipPlanEncoded", \
                                       "Children", "EstIncome", "Age", "AvgMonthlySpend", "CustomerSupportCalls"], outputCol="features")

In [ ]:
# instantiate the algorithm, take the default settings
rf=RandomForestClassifier(labelCol="label", featuresCol="features")

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)

pipeline = Pipeline(stages=[SI1,SI2,SI3,SI4,SI5,labelIndexer, assembler, rf, labelConverter])

In [ ]:
# Split data into train and test datasets
train, test = data.randomSplit([0.7,0.3], seed=3)
train.cache()
test.cache()

In [ ]:
# Build model
model = pipeline.fit(train)

In [ ]:
model.transform(test)

### Step 6: Score the test data set

In [ ]:
results = model.transform(test)
results=results.select(results["ID"],results["CHURN"],results["label"],results["predictedLabel"],results["prediction"],results["probability"])
results.toPandas().head(6)

### Step 7: Model Evaluation 

In [ ]:
print('Precision model1 = {:.2f}.'.format(results.filter(results.label == results.prediction).count() / float(results.count())))

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label", metricName="areaUnderROC")
print('Area under ROC curve = {:.2f}.'.format(evaluator.evaluate(results)))

We have finished building and testing a predictive model. The next step is to deploy it for real time scoring. 

###  Step 8:  Tune the model to find the best model

#### 8.1 Build a Parameter Grid specifying the parameters to be evaluated to determine the best combination

In [ ]:
# set different levels for the maxDepth
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = (ParamGridBuilder().addGrid(rf.maxDepth,[3,5,6]).build())

#### 8.2 Create a cross validator to tune the pipeline with the generated parameter grid
Cross-validation attempts to fit the underlying estimator with user-specified combinations of parameters, cross-evaluate the fitted models, and output the best one.

In [ ]:
# perform 3 fold cross validation
cv = CrossValidator().setEstimator(pipeline).setEvaluator(evaluator).setEstimatorParamMaps(paramGrid).setNumFolds(3)

In [ ]:
# train the model
cvModel = cv.fit(train)

# pick the best model
best_rfModel = cvModel.bestModel

In [ ]:
# score the test data set with the best model
cvresult=best_rfModel.transform(test)
cvresults_show=cvresult.select(cvresult["ID"],cvresult["CHURN"],cvresult["Label"],cvresult["predictedLabel"],cvresult["prediction"],cvresult["probability"])
cvresults_show.toPandas().head()

In [ ]:
print('Model Accuracy of the best fitted model = {:.2f}.'.format(cvresult.filter(cvresult.label == cvresult.prediction).count()/ float(cvresult.count())))
print('Model Accuracy of the default model = {:.2f}.'.format(results.filter(results.label == results.prediction).count() / float(results.count())))
print('   ')
print('Area under the ROC curve of best fitted model = {:.2f}.'.format(evaluator.evaluate(cvresult)))
print('Area under the ROC curve of the default model = {:.2f}.'.format(evaluator.evaluate(results)))

### Step 9: Save Model in the Project 

In [ ]:
from ibm_watson_machine_learning import APIClient

# IMPORTANT
# Replace with your Cloud API key and location
api_key = 'paste-your-key-here'
location = 'paste-your-location-here'  # For example, Dallas location is 'https://us-south.ml.cloud.ibm.com'

wml_credentials = {
    "apikey": api_key,
    "url": location
}

client = APIClient(wml_credentials)

In [ ]:
# IMPORTANT: Replace with your project id - copied from Insert Project Token cell
project_id = 'replace-with-your-project-id'
client.set.default_project(project_id)

In [ ]:
model_name = 'Predict Customer Churn-pyspark'
software_spec_uid = client.software_specifications.get_uid_by_name('spark-mllib_2.4')

model_props = {
    client.repository.ModelMetaNames.NAME: model_name,
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    client.repository.ModelMetaNames.TYPE: "mllib_2.4"
}

stored_model_details = client.repository.store_model(model=best_rfModel, pipeline=pipeline, meta_props=model_props, training_data=train)

**Author: Sidney Phoon and Elena Lowery**<br/>
Last updated: December 23, 2020